<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/GEMINI3_GENIE3_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import json
from google import genai
from google.genai import types

# --- API Key Setup (Using User's Logic) ---
# Set the API key using the user's preferred multi-source retrieval logic
GEMINI_API_KEY = None
try:
    # Attempt to load from Colab secrets ('GEMINI')
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI')
except (ImportError, KeyError):
    # Fallback to standard environment variable
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

# Define ThinkingLevel manually since the enum may not be directly exposed
class ThinkingLevel:
    LOW = "LOW"
    HIGH = "HIGH"

REQUESTED_MODEL_ID = 'gemini-3-pro-preview'

if not GEMINI_API_KEY:
    raise ValueError(
        "API Key not found. Please set the 'GEMINI' secret in Colab or "
        "the 'GEMINI_API_KEY' environment variable."
    )

# --- 1. THE SIMULATION ENVIRONMENT (The Genie 3 STAND-IN) ---

# Define the World Model (WM) as a Python Function/Tool
def run_simulation(scene_description: str, agent_action: str) -> str:
    """
    Simulates a complex action in a 3D environment and checks the outcome against physics.

    Args:
        scene_description: A detailed description of the starting environment.
        agent_action: The precise action the agent (robot) intends to take.

    Returns:
        A string describing the result of the simulation (SUCCESS or FAILURE).
    """
    print(f"\n--- [SIMULATION STARTED: Genie 3 WM] ---")
    print(f"Scene: {scene_description}")
    print(f"Action: {agent_action}")

    # --- The internal "World Model" logic (Simulated Physics Check) ---
    if "glass bottle" in scene_description and "vertical pinch grip" not in agent_action.lower():
        # This condition forces a failure to demonstrate the LLM revising its plan.
        return "SIMULATION FAILED: The narrow grasp was unstable due to the bottle's smooth surface. The glass bottle slipped and broke on the counter. Plan needs revision."
    elif "glass bottle" in scene_description and "vertical pinch grip" in agent_action.lower():
        return "SIMULATION SUCCESS: The vertical pinch grip maintained stability, compensating for the smooth surface. Bottle successfully placed on the counter. Plan is safe."
    else:
        return "SIMULATION SUCCESS: Action completed as planned."

# --- 2. THE LLM AGENT (The Gemini 3 Planner) ---

try:
    # Initialize client using the retrieved API key
    client = genai.Client(api_key=GEMINI_API_KEY)
except Exception as e:
    print(f"Error initializing Gemini client: {e}")
    exit()

def run_agentic_workflow(task_prompt: str):
    """Executes the agentic loop: Gemini 3 plans, calls Genie 3 to validate, and revises if necessary."""

    # 1. Provide the LLM with the task and the tool
    initial_prompt_text = f"TASK: {task_prompt}\n\nYour goal is to successfully complete this task. You must use the 'run_simulation' tool to validate your final plan before confirming success. You have a maximum of 3 attempts to simulate a successful action. Based on the task description, you MUST include a precise grasp type in your agent_action for the simulation."

    prompt_history = [
        types.Content(role="user", parts=[
            types.Part(text=initial_prompt_text)
        ])
    ]

    print("--- [GEMINI 3 (LLM) PLANNING STARTED] ---")
    print(f"Initial Task: {task_prompt}")

    # The loop allows the LLM to get simulation results and revise its plan (the Agentic Workflow)
    for attempt in range(1, 4):
        print(f"\n--- Attempt {attempt}/3 ---")

        # 2. LLM generates content, deciding whether to call the tool

        # FIX 2 & 4 APPLIED: Using client.models.generate_content AND moving 'tools' into 'config'
        response = client.models.generate_content(
            model=REQUESTED_MODEL_ID,
            contents=prompt_history,
            config=types.GenerateContentConfig(
                # FIX 4 APPLIED: The tools argument is now inside the config object
                tools=[run_simulation],
                # FIX 3 APPLIED: Nesting thinking_level inside types.ThinkingConfig
                thinking_config=types.ThinkingConfig(
                    thinking_level=ThinkingLevel.HIGH
                )
            )
        )

        # 3. Check if the LLM decided to call the simulation tool
        if response.function_calls:
            function_call = response.function_calls[0]

            # Extract arguments and run the local Python function (the "Genie 3" call)
            if function_call.name == "run_simulation":
                print(f"LLM calls run_simulation tool with its proposed action.")

                # Execute the simulated world model
                sim_args = dict(function_call.args)
                sim_result = run_simulation(**sim_args)

                # 4. Feed the result back to the LLM for re-evaluation

                # Add the function call and its result to the history so the LLM has context
                prompt_history.append(types.Content(
                    role="model",
                    parts=[types.Part.from_function_call(function_call)]
                ))
                prompt_history.append(types.Content(
                    role="tool",
                    parts=[types.Part.from_function_response(name="run_simulation", response=sim_result)]
                ))

                print(f"--- [SIMULATION ENDED] ---\nFeedback to LLM: {sim_result}")

                # If the simulation succeeded, the loop continues one more time to let the LLM generate the final text conclusion
                if "SUCCESS" in sim_result:
                    print("\nSimulation SUCCESS. Allowing LLM to provide final confirmation.")
                    continue

                # If the simulation failed, the LLM will use the feedback in the next loop
                continue

        # 5. If the model did not call the tool or successfully finished the simulation loop, print the final answer
        print("\n--- [GEMINI 3 FINAL ANSWER] ---")
        print(response.text)
        return

    print("\n--- [WORKFLOW FAILED] ---")
    print("Maximum attempts reached without a successful simulation.")

# --- 3. EXECUTE THE DEMO ---

# The task that requires the LLM to plan, simulate, encounter failure, and revise.
COMPLEX_TASK = "Retrieve the glass bottle of olive oil from the high shelf and place it on the kitchen counter without breaking it."

run_agentic_workflow(COMPLEX_TASK)

--- [GEMINI 3 (LLM) PLANNING STARTED] ---
Initial Task: Retrieve the glass bottle of olive oil from the high shelf and place it on the kitchen counter without breaking it.

--- Attempt 1/3 ---

--- [SIMULATION STARTED: Genie 3 WM] ---
Scene: A standard kitchen environment. A high wooden shelf (1.8m) holds a 750ml cylindrical glass bottle of olive oil. The bottle is full. A granite kitchen counter (0.9m) is located below and slightly forward of the shelf. The robot is a mobile manipulator with a multi-fingered soft gripper.
Action: Reach vertical alignment with the olive oil bottle on the high shelf. Open gripper. Approach bottle. Execute a 'Cylindrical Power Grasp' around the bottle's center of mass with 20N force. Lift vertically by 5cm to clear shelf. Retract arm slowly to minimize liquid sloshing. Lower arm to counter height. Place bottle on counter. Release grasp.

--- [SIMULATION STARTED: Genie 3 WM] ---
Scene: Kitchen environment. High shelf (1.8m) with a 750ml glass olive oil bo

## GEMIN4(POC)

In [9]:
import os
import json
from google import genai
from google.genai import types

# --- API Key Setup (Using User's Logic) ---
# Set the API key using the user's preferred multi-source retrieval logic
GEMINI_API_KEY = None
try:
    # Attempt to load from Colab secrets ('GEMINI')
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI')
except (ImportError, KeyError):
    # Fallback to standard environment variable
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

# Define ThinkingLevel manually
class ThinkingLevel:
    LOW = "LOW"
    HIGH = "HIGH"

REQUESTED_MODEL_ID = 'gemini-3-pro-preview'

if not GEMINI_API_KEY:
    # This check ensures the API key retrieval logic is working before we initialize the client.
    raise ValueError(
        "API Key not found. Please set the 'GEMINI' secret in Colab or "
        "the 'GEMINI_API_KEY' environment variable. Cannot proceed."
    )

# --- 1. INITIALIZE CLIENT ONCE ---
# We initialize the client explicitly with the retrieved key
try:
    client = genai.Client(api_key=GEMINI_API_KEY)
except Exception as e:
    print(f"Error initializing Gemini client: {e}")
    exit()

# --- 2. THE HOPE/NESTED LEARNING (NL) SIMULATED MEMORY ---

# This dictionary simulates the "Slow Learning Layer" or Long-Term Memory (LTM).
LONG_TERM_MEMORY = {
    "GLASS_BOTTLE_GRASP_STRATEGY": "UNINITIALIZED"
}

# --- 3. THE SIMULATION ENVIRONMENT (The Genie 3 STAND-IN) ---

def run_simulation(scene_description: str, agent_action: str) -> str:
    """
    Simulates a complex action in a 3D environment and checks the outcome against physics.

    Returns: A string describing the result and updates the LTM upon success.
    """
    print(f"\n--- [SIMULATION STARTED: Genie 3 WM] ---")
    print(f"Action: {agent_action}")

    # --- The internal "World Model" logic (Physics Check) ---
    is_glass_bottle = "glass bottle" in scene_description.lower()
    action_lower = agent_action.lower()

    # Define successful strategies that mitigate the primary risk (slippage)
    is_stable_grasp = (
        "vertical pinch grip" in action_lower or
        "towel" in action_lower or
        "two-handed" in action_lower or
        "stabilizing" in action_lower or
        "firmly grasp" in action_lower
    )

    # Define failure condition: bottle is glass AND no explicit stabilization method is used
    if is_glass_bottle and not is_stable_grasp:
        # Failure logic: Bottle breaks
        return "SIMULATION FAILED: Narrow or generic grasp was unstable. The glass bottle slipped and broke. Plan needs revision."

    elif is_glass_bottle and is_stable_grasp:
        # Success logic: Successful grip is recorded in LTM (Simulating HOPE's consolidation)

        # We will extract the specific successful strategy for LTM storage
        if "vertical pinch grip" in action_lower:
            strategy = "VERTICAL PINCH GRIP"
        elif "towel" in action_lower:
            strategy = "GRASP WITH TOWEL FOR FRICTION"
        elif "two-handed" in action_lower:
            strategy = "TWO-HANDED GRASP (NECK AND BASE)"
        else:
            strategy = "GENERAL STABLE GRASP (CONFIRMED SAFE)"

        LONG_TERM_MEMORY["GLASS_BOTTLE_GRASP_STRATEGY"] = strategy
        return f"SIMULATION SUCCESS: Stable grasp method ({strategy}) maintained stability. Plan is safe."

    else:
        return "SIMULATION SUCCESS: Action completed as planned."

# --- 4. THE LLM AGENT (The Gemini 4 Planner with LTM Access) ---

def run_gemini_4_agentic_workflow(task_prompt: str):
    """Executes the agentic loop, incorporating Long-Term Memory (HOPE)."""

    # 1. Access the HOPE/NL Long-Term Memory BEFORE planning
    ltm_status = LONG_TERM_MEMORY.get("GLASS_BOTTLE_GRASP_STRATEGY")

    # Create the initial prompt, feeding the LTM knowledge directly to the LLM
    ltm_prompt_part = ""
    if ltm_status != "UNINITIALIZED":
        # The LLM knows its core LTM has been updated by past experience (HOPE)
        ltm_prompt_part = f"\n\n***LONG-TERM MEMORY (LTM) ALERT:*** Past simulations have confirmed that the safe grasp strategy for glass bottles is the '{ltm_status}'. Incorporate this learned, safe grasp immediately into your first plan. DO NOT ATTEMPT ANY OTHER GRASP."

    initial_prompt_text = f"TASK: {task_prompt}{ltm_prompt_part}\n\nYour goal is to successfully complete this task. You must use the 'run_simulation' tool to validate your final plan before confirming success. Maximum 3 attempts."

    prompt_history = [
        types.Content(role="user", parts=[
            types.Part(text=initial_prompt_text)
        ])
    ]

    print("--- [GEMINI 4 (LLM w/ HOPE) PLANNING STARTED] ---")
    print(f"LTM Status: {ltm_status}")

    for attempt in range(1, 4):
        print(f"\n--- Attempt {attempt}/3 ---")

        # LLM generates content
        response = client.models.generate_content(
            model=REQUESTED_MODEL_ID,
            contents=prompt_history,
            config=types.GenerateContentConfig(
                tools=[run_simulation],
                thinking_config=types.ThinkingConfig(
                    thinking_level=ThinkingLevel.HIGH
                )
            )
        )

        # Check if the LLM called the simulation tool
        if response.function_calls:
            function_call = response.function_calls[0]

            if function_call.name == "run_simulation":
                print(f"LLM calls run_simulation tool with its proposed action.")

                sim_args = dict(function_call.args)
                sim_result = run_simulation(**sim_args)

                # Update history
                prompt_history.append(types.Content(
                    role="model",
                    parts=[types.Part.from_function_call(function_call)]
                ))
                prompt_history.append(types.Content(
                    role="tool",
                    parts=[types.Part.from_function_response(name="run_simulation", response=sim_result)]
                ))

                print(f"--- [SIMULATION ENDED] ---\nFeedback to LLM: {sim_result}")

                if "SUCCESS" in sim_result:
                    print("\nSimulation SUCCESS. The knowledge has been consolidated into LTM.")
                    continue

                continue

        # Final Answer (Only reached after a successful simulation or max attempts)
        print("\n--- [GEMINI 4 FINAL ANSWER] ---")
        print(response.text)
        return

    print("\n--- [WORKFLOW FAILED] ---")

# --- 5. EXECUTE THE DEMO TWICE TO SHOW LEARNING ---

COMPLEX_TASK = "Retrieve the glass bottle of olive oil from the high shelf and place it on the kitchen counter without breaking it."

print("==================================================================")
print("                       RUN 1: INITIAL LEARNING")
print("==================================================================")
run_gemini_4_agentic_workflow(COMPLEX_TASK)

# Check the LTM status after Run 1
print(f"\n*** LONG-TERM MEMORY (LTM) STATUS AFTER RUN 1: {LONG_TERM_MEMORY.get('GLASS_BOTTLE_GRASP_STRATEGY')} ***")

print("\n\n==================================================================")
print("                RUN 2: DEMONSTRATING HOPE/NL IMPACT")
print("==================================================================")
# The LTM state persists between runs, simulating the self-modifying memory
run_gemini_4_agentic_workflow(COMPLEX_TASK)

print(f"\n*** LTM STATUS AFTER RUN 2: {LONG_TERM_MEMORY.get('GLASS_BOTTLE_GRASP_STRATEGY')} ***")

                       RUN 1: INITIAL LEARNING
--- [GEMINI 4 (LLM w/ HOPE) PLANNING STARTED] ---
LTM Status: UNINITIALIZED

--- Attempt 1/3 ---

--- [SIMULATION STARTED: Genie 3 WM] ---
Action: Walk to the step stool, pick it up, and place it in front of the high shelf. Climb onto the step stool. Reach up with the right hand to grasp the body of the olive oil bottle firmly. Lift the bottle off the shelf, bring it down close to the chest, step down from the stool, and gently place the bottle on the kitchen counter.

--- [SIMULATION STARTED: Genie 3 WM] ---
Action: Place the step stool firmly in front of the shelf. Climb up. Reach out with the right hand to grip the neck of the bottle while simultaneously cupping the bottom of the bottle with the left hand to support the weight. Lift the bottle carefully off the shelf, keeping it upright. Hold it close to the body. Slowly step down from the stool one foot at a time. Extend arms gently to set the bottle down flat on the granite counter.

